# <center> IGR204 - Projet TimeX - 2017/2018

In [1]:
import numpy as np
import pandas as pd
idx = pd.IndexSlice

In [2]:
# Fonctions permettant la conversions des temps depuis "hh:mm" (String) vers leur équivalent en minutes (int)
def stringToMinutes(stringTime):
    assert ":" in stringTime, "char ':' not in given parameter"
    if stringTime == ":":
        return None
    hours, minutes = stringTime.split(":", maxsplit=1)
    return int(hours)*60 + int(minutes)

# Fonctions inverse
def minutesToString(intTime):
    assert isinstance(intTime, int), "parameter must be an integer"
    hours, minutes = divmod(intTime, 60)
    return "{}:{:02d}".format(hours, minutes)

In [3]:
# Lecture des données
data_filename = "./data/original_data.csv"

df = pd.read_csv(data_filename,
                 #na_values=":",
                 #converters={'Value':stringToMinutes},
                 index_col=['ACL00', 'ACL00_LABEL', 'TIME', 'SEX', 'SEX_LABEL',
                            'DAYSWEEK', 'DAYSWEEK_LABEL', 'GEO', 'GEO_LABEL'])

df = df.drop(['UNIT', 'UNIT_LABEL'], axis=1)

df['Minutes'] = df['Value'].apply(stringToMinutes)
df = df.drop(['Value'], axis=1)

df  #.isna().sum().sum()

Minutes
ACL00   ACL00_LABEL          TIME SEX SEX_LABEL DAYSWEEK DAYSWEEK_LABEL       GEO GEO_LABEL                                                
TOTAL   Total                2000 T   Total     D1-7     All days of the week BE  Belgium                                            1440.0
                                                                              BG  Bulgaria                                           1440.0
                                                                              DE  Germany (until 1990 former territory of the FRG)   1440.0
                                                                              EE  Estonia                                            1440.0
                                                                              EL  Greece                                                NaN
                                                                              ES  Spain                                              1440.0
                                                                              FR  France                                             1440.0
                                                                              IT  Italy                                              1440.0
                                                                              LV  Latvia                                             1440.0
                                                                              LT  Lithuania                                          1440.0
                                                                              LU  Luxembourg                                            NaN
                                                                              HU  Hungary                                               NaN
                                                                              NL  Netherlands                                           NaN
                                                                              AT  Austria                                               NaN
                                                                              PL  Poland                                             1440.0
                                                                              RO  Romania                                               NaN
                                                                              SI  Slovenia                                           1440.0
                                                                              FI  Finland                                            1440.0
                                                                              UK  United Kingdom                                     1440.0
                                                                              NO  Norway                                             1440.0
                                                                              RS  Serbia                                                NaN
                                                D1-5     Monday to Friday     BE  Belgium                                            1440.0
                                                                              BG  Bulgaria                                           1440.0
                                                                              DE  Germany (until 1990 former territory of the FRG)   1440.0
                                                                              EE  Estonia                                            1440.0
                                                                              EL  Greece                                                NaN
                                                                              ES  Spain                                              1440.0
                                                                        

## Extract Countries & Activities List

In [4]:
countries = list(df.index.get_level_values(8).unique().values)
activities_Code = list(df.index.get_level_values(0).unique().values)

print("Nb Countries =", len(countries))
print("Nb Activities =", len(activities_Code))

Nb Countries = 21
Nb Activities = 56


In [5]:
## Vérification qu'il y a bien 21 valeurs (pour les 21 pays) par filtrage
for year in [2000, 2010]:
    for gender in ['M', 'F', 'T']:
        for daysweek in ['D1-5', 'D6-7', 'D1-7']:
            for acl in activities_Code:
                nb_valeurs = len(df.loc[idx[acl, :, year, gender, :, daysweek], :].values)
                assert nb_valeurs==21, "!! Il y a {} valeurs pour Y={}, gender={}, day={}".format(nb_valeurs,
                                                                                                  year,
                                                                                                  gender,
                                                                                                  daysweek)


## General Function to Compute GHI

In [6]:
year=2000
gender='T'
daysweek='D1-7'

total_time_constraint = np.zeros(21)

sleep = df.loc[idx['AC01', :, year, gender, :, daysweek], :].values
eating = df.loc[idx['AC02', :, year, gender, :, daysweek], :].values

total_time_constraint = sleep + eating
total_time_constraint

array([[614.],
       [667.],
       [597.],
       [580.],
       [ nan],
       [620.],
       [665.],
       [612.],
       [609.],
       [600.],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [602.],
       [ nan],
       [590.],
       [588.],
       [588.],
       [565.],
       [ nan]])

In [7]:
# ['ACL00', 'ACL00_LABEL', 'TIME', 'SEX', 'SEX_LABEL', 'DAYSWEEK', 'DAYSWEEK_LABEL', 'GEO', 'GEO_LABEL']

def computeGHI_allCountries(constraint_ACLs, leisure_ACLs, year=2000, gender='T', daysweek='D1-7'):
    
    assert year in [2000, 2010], "year must be an integer equal to 2000 or 2010"
    assert gender in ['M', 'F', 'T'], "gender must be a String equal to 'M', 'F' or 'T'"
    assert daysweek in ['D1-5', 'D6-7', 'D1-7'], "daysweek must be a String equal to 'D1-5', 'D6-7' or 'D1-7'"
    
    total_time_constraint = np.zeros(21)
    total_time_leisure = np.zeros(21)
    idx = pd.IndexSlice
    
    for cons_acl in constraint_ACLs:
        values = df.loc[idx[cons_acl, :, year, gender, :, daysweek], :].values.reshape(-1)
        total_time_constraint += values
    
    for leis_acl in leisure_ACLs:
        total_time_leisure += df.loc[idx[leis_acl, :, year, gender, :, daysweek], :].values.reshape(-1)
    
    return total_time_constraint, total_time_leisure

In [8]:
# Par exemple :
constraint_ACLs = ['AC0', 'AC01', 'AC02', 'AC03']
leisure_ACLs = ['AC9B', 'AC936', 'AC938']
# ------------------------------------------------

total_time_constraint, total_time_leisure = computeGHI_allCountries(constraint_ACLs, leisure_ACLs,
                                                                    year=2000, gender='T', daysweek='D1-7')

### Display in a Dataframe

In [9]:
res = pd.DataFrame({"Tot_Leis":total_time_leisure, "Tot_Cons":total_time_constraint}, index=countries)
res['GHI'] = res['Tot_Leis'] / res['Tot_Cons']
res

,Tot_Cons,Tot_Leis,GHI
Belgium,1316.0,22.0,0.016717
Bulgaria,1409.0,14.0,0.009936
Germany (until 1990 former territory of the FRG),1299.0,23.0,0.017706
Estonia,1266.0,17.0,0.013428
Greece,NaN,NaN,NaN
Spain,1336.0,16.0,0.011976
France,1418.0,NaN,NaN
Italy,1347.0,21.0,0.015590
Latvia,1299.0,22.0,0.016936
Lithuania,1309.0,19.0,0.014515


In [10]:
activities_Code = list(df.index.get_level_values(0).unique().values)
activities_label = list(df.index.get_level_values(1).unique().values)
df_activities = pd.DataFrame({"ACL":activities_Code, "Label":activities_label})
df_activities
#df_activities.to_excel("activities_TimeUse.xlsx")

,ACL,Label
0,TOTAL,Total
1,AC0,Personal care
2,AC01,Sleep
3,AC02,Eating
4,AC03,Other and/or unspecified personal care
5,AC1_TR,"Employment, related activities and travel as p..."
6,AC1A,Main and second job and related travel
7,AC1B,Activities related to employment and unspecifi...
8,AC2,Study
9,AC21A,School and university except homework


### HELP:
> Si on a plusieurs activités + plusieurs pays en multiindex,  
et qu'on veut mettre un des deux en colonnes :

```python
df.unstack(level=...)
```